# Kaggle Halite Episode Service API Client

__Purpose is to make an API and program to fetch data from Kaggle__

## Kaggle Halite API Endpoints

Kaggle's Sam Harris release an API to pull Halite match data from games hosted on Kaggle.
- Post documenting API: https://www.kaggle.com/c/halite/discussion/164932
- API Wrapper here: https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/api.py
- API service is located at: https://www.kaggle.com/requests/EpisodeService/

#### GetEpisodeReplay Endpoint

- URL: 'https://www.kaggle.com/requests/EpisodeService/GetEpisodeReplay'
- Verb: Post
- Schema: JSON
- JSON Payload: {"EpisodeId": episode_id} # int

#### ListEpisodes Endpoint

- URL: 'https://www.kaggle.com/requests/EpisodeService/ListEpisodes'
- Verb: Post
- Schema: JSON
- JSON Payload:
    - {"Ids": episode_ids} # List[int]
    - {"TeamId": team_id} # int
    - {"SubmissionId": submission_id} # int

## Kaggle Halite API Client Implementation

In [48]:
__version__ = '0.0.1'


from urllib.parse import urljoin
import inspect
import requests

class KaggleClient(object):
    """ A Kaggle Halite Episode Service API wrapper. """
    
    def __init__(self):
        self.requester = self.Requester()
        self._attach_endpoints()
        
    def _attach_endpoints(self):
        """ Generate and attach endpoints """
        for name, endpoint in inspect.getmembers(self):
            if (inspect.isclass(endpoint) 
                and issubclass(endpoint, self._Endpoint) 
                and endpoint is not self._Endpoint):
                endpoint_instance = endpoint(self.requester)
                setattr(self, endpoint.name, endpoint_instance)
    
    class Requester(object):
        """ An object for making API requests """
        
        def GET(self, url, params=None, json=None):
            if None is params:
                params = dict()
            if None is json:
                json = dict()
    
            params.setdefault('datatype', 'json')
            response = requests.post(url, params=params, json=json)
            if 200 != response.status_code:
                error = 'HTTPError: {}'.format(response.status_code)
                return {'success': False, 'error': error}
            try:
                return response.json()
            except ValueError as err:
                return {'success': False, 'error': err}
        
    class _Endpoint(object):
        """ Base class of an endpoint """
        url = 'https://www.kaggle.com/requests/EpisodeService/'
        
        def __init__(self, requester):
            self.requester = requester            
        
        def _GET(self, path, params=None, json=None):
            request_url = urljoin(self.url, path)
            #print(f'url: {self.url}, request_url: {request_url}, path: {path}')
            return self.requester.GET(request_url, params, json)
    
    class Replay(_Endpoint):
        name = 'replay'
        
        def episode(self, episode_id, params=None):
            json = {"EpisodeId": episode_id}
            return self._GET('GetEpisodeReplay', params, json)
        
    class Episodes(_Endpoint):
        name = 'episodes'

        def episodes(self, episode_ids, params=None):
            json = {'Ids': episode_ids}
            return self._GET('ListEpisodes', params, json)
        
        def team(self, team_id, params=None):
            json = {'TeamId': team_id}
            return self._GET('ListEpisodes', params, json)
        
        def submission(self, submission_id, params=None):
            json = {'SubmissionId': submission_id}
            return self._GET('ListEpisodes', params, json)

In [49]:
api = KaggleClient()

In [41]:
json = api.episodes.team(TEAM_ID)

In [42]:
json.keys()

dict_keys(['result', 'wasSuccessful'])

In [44]:
json['result'].keys()

dict_keys(['episodes', 'teams'])

In [46]:
json['result']['episodes'][0].keys()

dict_keys(['id', 'competitionId', 'createTime', 'endTime', 'replayUrl', 'adminNotes', 'state', 'type', 'agents'])